# Laboratorio#2: Modelo Vectorial de Recuperación de Información

El Modelo  Vectorial de Recuperación de Información (MVRI) es un modelo algebraico para representar documentos de texto (o más generalmente, elementos) como vectores, de modo que la distancia entre vectores representa la relevancia entre los documentos. Es fundamental para una serie de operaciones de recuperación de información (RI), incluida la puntuación de documentos en una consulta, la clasificación de documentos y la agrupación de documentos.

Durante la clase se estará trabajando con el Modelo Vectorial. Este de define como:
- **D**:  Vectores de pesos no binarios asociados a los términos de los documentos.
- **Q**: Vectores de pesos no binarios asociados a los términos de la consulta.
- **F**: Espacio n-dimensional y operaciones entre vectores del Álgebra Lineal.
- **R**: $sim(d_j, q) = 
		 		\displaystyle{\frac{\sum_{i=1}^{n} w_{i,j} \times w_{i,q}}{\sqrt{\sum_{i=1}^{n} w_{i,j}^{2}}\times \sqrt{\sum_{i=1}^{n} w_{i,q}^{2}}}}
		 	$

In [2]:
# Configurando el entorno

# Fuente del corpus
import ir_datasets

# Facilita el trabajo con los términos indexados del corpus
from gensim.corpora import Dictionary

# Para realizar el proceso de reduccion de dimensiones
from sklearn.decomposition import PCA

# Funciones útiles auxiliares
from teacher_help import tokenize, cosine_similarity

# Facilita la creación de vectores y matrices de gran dimensión
import numpy as np

# Facilita el trabajo con funciones matemáticas
import math

### Cranfield

El corpus Cranfield es un conjunto clásico de datos en el campo de la Recuperación de Información compuesto por aproximadamente 1,400 resúmenes de artículos de investigación en aerodinámica. Cada documento en este corpus incluye un título, un resumen conciso del contenido, y en algunos casos, palabras clave y referencias bibliográficas. Acompañando a estos documentos, hay alrededor de 225 consultas de prueba y sus correspondientes juicios de relevancia, proporcionados por expertos, que indican la pertinencia de cada documento para una consulta específica. Este diseño estructurado y su enfoque específico en temas de aerodinámica hacen del corpus una herramienta esencial para evaluar la eficacia de Sistemas de Recuperación de Información (SRI), sirviendo como un modelo estándar para pruebas comparativas y consistentes en esta disciplina.

La variable **dataset**, instancia de la clase **ir_datasets.datasets.base.Dataset**, tiene 3 funciones:
1. _docs_iter()_
2. _queries_iter()_
3. _qrels_iter()_

Pero durante esta clase, solo se trabajará con la primera de ellas. La función **docs_iter()** devuelve un objeto iterable de tuplas de dimensión 5, referentes a los documentos. Los campos de cada tupla son:
  - Identificador (str)
  - Título (str)
  - Texto (str)
  - Autor (str)
  - Referencia bibliográfica (str)


---

En el laboratorio anterior se vio, entre otras cosas, el proceso de la tokenización de documentos y la representación de los documentos según BoW (Bag Of Words, o bolsa de palabras). 

Nuestro primer objetivo es ejercitar los contenidos antes mencionados y preparar los datos para su posterior uso. Además veremos el proceso de generación del vocabulario.

### Ejercicio 1: Prepare los datos para poder trabajar.
Para ello, 

a) Cree el vocabulario del conjunto de datos.

b) Convierta a la representación **bolsa de palabras** cada documento.

###### Ayuda: Considere usar la clase `Dictionary`.

In [ ]:
# TODO

En el caso de la representación de los documentos en el Modelo Vectorial, tanto los documentos como las consultas se representan como vectores no binarios, donde cada dimensión indica el peso de un término o *token* independiente en el documento. Se han desarrollado varias formas diferentes de calcular estos valores, y uno de los esquemas más conocidos es la ponderación *tf-idf*.

El *tf–idf* es el producto de dos estadísticas: la *frecuencia de los términos* ($tf_{i,j}$) y la *frecuencia inversa de los documentos* ($idf_i$). Siendo $w_{i,j}$ el peso de una palabra en documento se cumple que:
$$
w_{i,j} = tf_{i,j} * idf_i
$$



La *frecuencia nromalizada del término* $t_i$ en el documento $d_j$ se define como:

$$
tf_{i,j} = \frac{freq_{i,j}}{max\_freq_{i,j}} = \alpha + (1 - \alpha) \cdot \frac{freq_{i,j}}{max\_freq_{i,j}}
$$

Donde:
- $freq_{i,j}$: Frecuencia del término $t_i$ en el documento $d_j$.
- $max\_freq_{i,j}$: Mayor frecuencia entre todos los términos del documento $d_j$.
- $\alpha \in [0, 1]$: Término de suavizado que amortigua la contribución de la frecuencia del término. 
Si el término no aparece en el documento ($t_i \notin d_j$), entonces $tf_{i,j} = 0$

La *frecuencia inversa de los documentos* está dada por:

$$
idf_i = \log\left(\frac{N}{n_i}\right)
$$

Donde:
- $N$: Número total de documentos en el corpus.
- $n_i$: Número de documentos que contienen el término $t_i$.

---

### Ejercicio 2:
Obtenga la representación **tf-idf** de cada elemento del corpus y de la consulta definida. Para esto:
- 1. Implemente la clase TfidfModel.
- 2. Cree un objeto de la clase implementada pasandole el corpues previamente construído.
###### Ayuda: Vea la clase TfidfModel de gensim.models.

In [ ]:
class TfidfModel:
    def __init__(self, corpus_bow, dictionary, alpha=0.0):
        """
        Inicializa el modelo tf-idf y calcula los IDF globales.

        Args:
            corpus_bow: Corpus en formato BoW (lista de [(term_id, freq), ...]).
            dictionary: Objeto Dictionary de gensim.
            alpha: Parámetro de suavizado (opcional).
        """
        self.dictionary = dictionary
        self.alpha = alpha
        self.num_docs = len(corpus_bow)
        self.corpus_bow = corpus_bow
        self.idf = self._compute_idf(corpus_bow)
        self.corpus_tfidf = self.corpus2dense(self.transform())

    def _compute_idf(self, corpus_bow):
        """
        Calcula la frecuencia inversa de documento (idf).
        
        Args:
            - corpus_bow : [[(int, int)]]
                Corpus represenatdo en bosla de palabras (bow).
        
        Return:
        - {int: float}

        """
        
        # TODO

    def _compute_tf(self, bow_doc):
        """
        Calcula tf normalizado con suavizado para un documento.
        
        Args:
            - bow_doc : [(int, int)]
                Documento representado en bolsa de palabras.
            
        Return:
        - {int: float}
        """
        
        # TODO
        

    def get_document_tfidf(self, bow_doc):
        """
        Devuelve tf-idf de un solo documento.
        
        Args:
            - bow_doc : [(int, int)]
                Documento representado en bolsa de palabras.
            
        Return:
        - [(int, float)]
        """
        
        # TODO

    def transform(self, corpus=None):
        """
        Devuelve el corpus transformado a tf-idf en una matriz dispersa.
        
        Args:
            - corpus : [(int, int)]
                Corpus compuesto por los documentos representados en bolsa de palabras.
            
        Return:
        - [[(int, float)]]
        """
        
        # TODO

    def corpus2dense(self, tfidf_corpus):
        """
        Convierte el corpus tf-idf a una matriz densa [docs x terms].
        
        Args:
            - corpus : [(int, int)]
                Corpus compuesto por los documentos representados en bolsa de palabras.
            
        Return:
        - numpy.array(num_docs, num_terms)
        """
        
        # TODO


### Ejercicio 2.1:
Obtenga el tf-idf asociado a la query dada.

In [ ]:
query_text = 'what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .'

# TODO

### Ejercicio 3: Obtenga el ranking (decreciente) de los documentos que _satisfacen_ a la consulta previamente definida.

###### Ayuda: Considere usar la función `cosine_similarity`. 

In [ ]:
def retrieve_documents(corpus_matriz, vector_query):
    """
    Gets the similarity between the corpus and a query
    
    Args:
    - corpus_matriz : [[float]]
        tf-idf representation of the query. Each row is considered a document.
    - vector_query : [float]
        tf-idf representation of the query.
        
    Return:
    - [(int, float)]

    """
    
    # TODO

retrived_docs = retrieve_documents(tfidf.corpus_tfidf, query_vector[0])
print(retrived_docs[:10])
print(len(retrived_docs)) 

Uno de los problemas más comunes cuando se trabaja con datos muchas dimensiones es el conocido "maldición de la dimensionalidad". Este fenómeno entre otras cosas afecta nociones como distancia o similitud, haciendo que todos los objetos parezcan distantes y diferentes.

Debido a esto, una de las ideas básicas para evitar estos problemas consiste en aplicar técnicas de reducción de dimensiones. Este proceso consiste en la transformación de datos de un espacio de alta dimensión a un espacio de baja dimensión, pero en este nuevo espacio se intenta conservar algunas propiedades significativas de los datos originales, idealmente cercanas a su dimensión intrínseca.

Una de las principales técnicas lineales para la reducción de la dimensionalidad es el Análisis de Componentes Principales (PCA por sus siglas en inglés). Esta técnica realiza un mapeo lineal de los datos a un espacio de dimensiones inferiores de tal manera que se maximiza la varianza de los datos en la representación de dimensiones bajas.

### Ejercicio 4: Reduzca las dimensiones del corpus (vectores de tf-idf) y con los nuevos vectores recupere los documentos asociados a la consulta dada.
###### Ayuda: Considere usar la clase `PCA` y de ahí, las funciones `fit` y `transform`.

In [ ]:
# Índice de varianza
variance = 0.90

# TODO